In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Load finetuned model from hub

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Mr-TD/Llama-2-7b-hf-finetune-MOM_Summary-Points")
model = AutoModelForCausalLM.from_pretrained("Mr-TD/Llama-2-7b-hf-finetune-MOM_Summary-Points")

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_length=1000,do_sample=True,return_full_text=False)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_length=2000,do_sample=True,return_full_text=False)
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_length=4096,do_sample=True,return_full_text=False)

### Formate meeting transcript for summarization (Input)

In [6]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()

def generate_prompt(
    Transcript: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{Transcript.strip()}

### Response:
""".strip()

Transcript = """
{"date": "2023-03-08",
"time": "10:00 AM",
"attendess": ["Manager", "Junior 1", "Junior 2", "Junior 3"],
"transcript": 
"**Manager:** Good morning team. Let's start our daily standup meeting. Junior 1, could you please share your progress on the software development project?

**Junior 1:** Yesterday, I completed the design for the user interface and I'm currently working on implementing the backend functionality.

**Manager:** Great. Junior 2, what have you been working on?

**Junior 2:** I've been working on the database integration and I'm almost done with the data modeling.

**Manager:** That's good to hear. Junior 3, what's your status?

**Junior 3:** I've been focusing on the testing framework and I've set up the automated test cases.

**Manager:** Excellent. Are there any blockers or challenges that you're facing?

**Junior 1:** I'm having some trouble with the integration of the UI with the backend.

**Manager:** Okay. Let's discuss this after the meeting and we'll find a solution.

**Junior 2:** I'm a bit stuck on the data validation. The data coming from the UI doesn't seem to be formatted correctly.

**Manager:** I see. We can look into this together.

**Junior 3:** No blockers from my side.

**Manager:** That's good. Keep up the good work. Let's continue with our daily tasks and we'll touch base again tomorrow.

**Meeting adjourned."
}
"""

In [40]:
Transcript = """
{'date': '2023-03-08',
 'time': '10:00 AM - 12:00 PM',
 'attendess': ['Manager',
  'Junior Developer 1',
  'Junior Developer 2',
  'Junior Developer 3',
  'Junior Developer 4',
  'Junior Developer 5',
  'Technical Support Engineer'],
 'transcript': ["Manager:Good morning team, let's start our daily standup meeting.",
  'Junior Developer 1:Yesterday, I completed the feature implementation for task XYZ. Today, I plan to start working on bug fixes.',
  "Junior Developer 2:I've been working on refactoring the codebase. I've encountered some issues with unit tests, but I'm confident I can resolve them by this afternoon.",
  "Junior Developer 3:I've been working on the integration with the third-party API. I'm facing some challenges with the authentication process, but I'm exploring different solutions.",
  "Junior Developer 4:I've been writing documentation for the new feature. I'm almost done, and I plan to submit it for review by the end of the day.",
  "Junior Developer 5:I've been assisting users with technical issues. There's been a recent spike in support requests related to a specific feature. I'm working with the team to investigate the issue.",
  "Technical Support Engineer:I've been testing the new build and have identified a potential performance issue. I've reported it to the team, and we're looking into it.",
  'Manager:Thank you for the updates. Does anyone have any roadblocks or issues they need assistance with?',
  "Junior Developer 3:I'm stuck with the API authentication. I've tried multiple approaches, but none of them seem to work.",
  "Technical Support Engineer:I've worked with this API before. Let me take a look and see if I can help.",
  "Manager:That's great. Thanks for stepping in. Junior Developer 3, please work with the Technical Support Engineer to resolve this issue.",
  'Junior Developer 3:Thank you, I appreciate the assistance.',
  'Manager:Are there any other updates or announcements?',
  "Junior Developer 2:I'd like to suggest we introduce unit testing for the new feature before it goes live. It will help us catch potential bugs early on.",
  "Manager:That's a great suggestion. We'll schedule a session to discuss this further and finalize the testing plan.",
  "Manager:Before we end, I'd like to remind everyone about the upcoming code review deadline. Please complete your reviews and provide feedback by the end of the week.",
  'Attendess:Understood.',
  "Manager:Thank you, team. Let's continue the hard work and keep each other updated on our progress. See you all tomorrow for our next meeting."]}
"""

### Run finetune model on formated data

In [41]:
prompt = generate_prompt(Transcript)
response = pipe(prompt)
response[0]['generated_text']

### Convert plain text to markdown format

In [46]:
import textwrap
from IPython.display import Markdown

## functions
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
to_markdown(response[0]['generated_text'])

In [ ]:
response[0]['generated_text'].split("###")[3]